In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import nltk
import nltk.corpus
import numpy as np
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
plt.rcParams["figure.figsize"] = [10, 8]

In [2]:
def tokenize_text(doc):
    """Combine the strings in the "response" column of dataframe df into one long string. Then, tokenize the
    string and make all words lowercase."""

    # Tokenize and make lowercase.
    words = nltk.word_tokenize(doc)
    words = [w.lower() for w in words]
    
    return words


def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN)


def lemmatize_text(words):
    """Lemmatize words to get the base words. The input 'words' is a list of of words."""
    
    lemmatizer = nltk.WordNetLemmatizer()
    word_tags = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(w, wordnet_pos(t)) for (w, t) in word_tags]
    
    return words


def remove_stopwords(words):
    """Remove stopwords from a string."""
    
    stopwords = nltk.corpus.stopwords.words("english")
    words = [w for w in words if w not in stopwords]
    
    return words

def clean_text(doc):    
    """Tokenize, lemmatize, and remove stopwords for the text of all articles."""
    
    words = re.sub("< ?/?[a-z]+ ?>|\n", "", doc)
    words = tokenize_text(words)
    words = lemmatize_text(words)
    words = remove_stopwords(words)
    doc = [w for w in words if w.isalnum()]
    doc = ' '.join(doc)
    
    return doc

def clean_df(df):
    """Combine the title and content of each post into one string and clean each string."""
    text = df['title'] + " " + df['content']
    df_clean = pd.DataFrame([clean_text(i) for i in text])
    df_clean.columns = ["text"]
    #df_clean["tags"] = df["tags"]
    df_clean = pd.concat([df_clean, pd.DataFrame(df["tags"])],axis = 1, sort = False)
    return df_clean

In [3]:
## Functions from examine_tags.ipynb
def get_top_tags(df):
    tag_list = [tags.split(' ') for tags in df['tags']]
    flat_list = [item for sublist in tag_list for item in sublist]
    fq = nltk.FreqDist(w for w in flat_list)
    df_fq = pd.DataFrame.from_dict(fq, orient="index").reset_index()
    df_95 = df_fq[df_fq.iloc[:,1] >= np.percentile(df_fq.iloc[:,1],95)].reset_index(drop=True)
    df_95.columns = ["term", "fq"]
    return df_95

def subset_top_df(df, top_tags_df):
    """
    df: DataFrame with all posts
    top_tags_df: Data frame of top tags
    """
    tags_list = [tags.split(' ') for tags in df['tags']]
    indeces = [i for i in range(len(tags_list))
               if list(set(top_tags_df['term']) & 
                       set(tags_list[i])) != []]
    return df.loc[indeces]

In [4]:
def trim_tags_clean(df):
    top = get_top_tags(df)
    top_subset = subset_top_df(df, top).reset_index()
    return clean_df(top_subset)

In [ ]:
# Stack exchange topic names
names = ["biology","cooking","crypto","diy","robotics","travel"]

def get_paths(name):
    """Get path names for each file."""
    path = "data/"+name+".csv"
    return path

# Get path names
paths = [get_paths(i) for i in names]

# All data frames in a list.
dfs = [pd.read_csv(i) for i in paths]

# Get a list of the cleaned data frames.
trim_clean_dfs = [trim_tags_clean(i) for i in dfs]

# Save cleaned dfs as csv
for i in range(len(names)):
    trim_clean_dfs[i].to_pickle(names[i]+"_trim_clean.dat")

In [3]:
data = pd.read_pickle("~/Downloads/robotics_finalDF")
data_all = pd.read_pickle("~/Downloads/full_finalDF")

In [ ]:
data_all

In [4]:
# test
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

train_text = train['text']
test_text = test['text']

vectorizer.fit(train_text)
vectorizer.fit(test_text)

x_train = vectorizer.transform(train_text)
#y_train = train.drop(labels = ['text'], axis=1)
y_train = train.drop(labels = ['text','tags'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['text', 'tags'], axis=1)

In [32]:
split_list = [tags.split(" ") for tags in list(data['tags'])]
split = [item for sublist in split_list for item in sublist]
unique = list(set(split))
categories = unique

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    #print("\n")
    #f1_score(test[category], prediction,average='weighted')

**Processing dh-parameters comments...**
Test accuracy is 0.9961538461538462
**Processing linear-bearing comments...**
Test accuracy is 1.0
**Processing soccer comments...**
Test accuracy is 1.0
**Processing motion comments...**
Test accuracy is 0.9942307692307693
**Processing simulator comments...**
Test accuracy is 0.9903846153846154
**Processing linux comments...**
Test accuracy is 0.9961538461538462
**Processing dead-reckoning comments...**
Test accuracy is 1.0
**Processing power comments...**
Test accuracy is 0.9788461538461538
**Processing failure comments...**
Test accuracy is 1.0
**Processing ugv comments...**
Test accuracy is 0.9980769230769231
**Processing h-bridge comments...**
Test accuracy is 0.9942307692307693
**Processing orientation comments...**
Test accuracy is 1.0
**Processing lithium-polymer comments...**
Test accuracy is 0.9961538461538462
**Processing protection comments...**
Test accuracy is 1.0
**Processing planning comments...**
Test accuracy is 0.9961538461538

KeyboardInterrupt: 

In [52]:
data

,text,tags,real-time,arm,gearing,power,dynamixel,manipulator,visual-servoing,routing,...,automation,path-planning,reprap,slam,robotc,roomba,linux,planning,wifi,computer-vision
0,right approach write spin controller soccer ro...,soccer control,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,modify low cost hobby servo run get hobby serv...,control rcservo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,good robotics project look starting point proj...,microcontroller arduino raspberry-pi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,good method tune process noise kalman filter o...,odometry localization kalman-filter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,choose right combination quadcopter many site ...,quadcopter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,mobile robot localization known map want local...,localization mobile-robot,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,algorithm use balance two wheel robot use gyro...,control gyroscope balance two-wheeled,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,accurate way obtain locational fix use gps obv...,localization gps,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,algorithm use construct map explored area use ...,slam localization gps mapping acoustic-rangefi...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,detect dc motor robot start fail characteristi...,sensors failure motor,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Read the one hot encoding dataframes
biology = pd.read_pickle("../one_hot_encoding/biology_finalDF")
cooking = pd.read_pickle("../one_hot_encoding/cooking_finalDF")
crypto = pd.read_pickle("../one_hot_encoding/crypto_finalDF")
diy = pd.read_pickle("../one_hot_encoding/diy_finalDF")
full = pd.read_pickle("../one_hot_encoding/full_finalDF")
robotics = pd.read_pickle("../one_hot_encoding/robotics_finalDF")
travel = pd.read_pickle("../one_hot_encoding/travel_finalDF")

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

train_text = train['text']
test_text = test['text']

vectorizer.fit(train_text)
vectorizer.fit(test_text)

x_train = vectorizer.transform(train_text)
y_train = train.drop(labels = ['text','tags'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['text', 'tags'], axis=1)

In [20]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train[::5], y_train[::5])

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(x_test,predictions))
print("f1-score = ",f1_score(x_test,predictions, average = 'weighted'))

TypeError: len() of unsized object

In [11]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

def br_fscore(x_train, y_train, x_test, y_test):
    # initialize binary relevance multi-label classifier
    # with a gaussian naive bayes base classifier
    classifier = BinaryRelevance(GaussianNB())

    # train
    classifier.fit(x_train, y_train)

    # predict
    predictions = classifier.predict(x_test)

    # accuracy
    #print("Accuracy = ",accuracy_score(y_test,predictions))
    #print("f1-score = ",f1_score(y_test,predictions, average = 'weighted'))
    return f1_score(y_test,predictions, average = 'weighted')

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

In [18]:
# using Label Powerset
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import accuracy_score, f1_score
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("f1_score = ",f1_score(y_test,predictions,average = "weighted"))
print("f1_score = ",f1_score(y_test,predictions,average = "micro"))

/Users/christinachang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/christinachang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy =  0.04807692307692308
f1_score =  0.06899302264142039
f1_score =  0.15268817204301074


/Users/christinachang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/christinachang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
